## Building Basic A Chatbot


In [1]:
import os
from dotenv import load_dotenv
load_dotenv() 

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020967AF76A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020967B3CA90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Abhi i am Software Engineer")])

AIMessage(content="Hi Abhi,\n\nIt's nice to meet you! I'm glad you introduced yourself.\n\nWhat kind of software engineering do you specialize in?  \n\nI'm always interested in learning more about different areas of software development.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 19, 'total_tokens': 70, 'completion_time': 0.092727273, 'prompt_time': 8.0029e-05, 'queue_time': 0.022561238, 'total_time': 0.092807302}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6dcbc284-625f-4bee-aed4-fd35836eec04-0', usage_metadata={'input_tokens': 19, 'output_tokens': 51, 'total_tokens': 70})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Abhi i am Software Engineer"),
        #AIMessage(content="Hi Abhi, it's nice to meet you! \n\nBeing a software engineer is a fascinating field. What kind of software do you specialize in?  \n\nI'm always eager to learn more about what people do.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You said your name is Abhi and you are a Software Engineer.  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 41, 'total_tokens': 60, 'completion_time': 0.034545455, 'prompt_time': 0.000558119, 'queue_time': 0.020674099, 'total_time': 0.035103574}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac736396-a505-4108-8193-b09fd2634d48-0', usage_metadata={'input_tokens': 41, 'output_tokens': 19, 'total_tokens': 60})

### Message History
 This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. 

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Abhi i am Software Engineer")],
    config=config
)

In [9]:
response.content

"Hi Abhi, it's nice to meet you!\n\nIt's great that you're a software engineer. What kind of software do you work on?  \n\nIs there anything I can help you with today? Perhaps you have a coding question, need to brainstorm ideas, or just want to chat about technology? 😊  \n\n"

In [10]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Abhi i am Software Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Abhi, it's nice to meet you!\n\nIt's great that you're a software engineer. What kind of software do you work on?  \n\nIs there anything I can help you with today? Perhaps you have a coding question, need to brainstorm ideas, or just want to chat about technology? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 19, 'total_tokens': 90, 'completion_time': 0.129090909, 'prompt_time': 8.212e-05, 'queue_time': 0.021816206999999997, 'total_time': 0.129173029}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-34529adf-d235-4d10-b692-29e32f05a306-0', usage_metadata={'input_tokens': 19, 'output_tokens': 71, 'total_tokens': 90})])}

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Abhi. 😊  \n\nI remembered from our first conversation! \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 103, 'total_tokens': 123, 'completion_time': 0.036363636, 'prompt_time': 0.003308084, 'queue_time': 0.022583123, 'total_time': 0.03967172}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ded7564-8918-436a-8f19-d5b8073b0c6a-0', usage_metadata={'input_tokens': 103, 'output_tokens': 20, 'total_tokens': 123})

In [12]:

config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it!\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Rohit")],
    config=config1
)
response.content

'Nice to meet you, Rohit!  \n\nIs there anything I can help you with today?\n'

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Rohit! 😊  \n\nI remember that you told me earlier. \n\nWhat can I do for you, Rohit?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. 

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhi")]})

AIMessage(content="Hi Abhi! 👋 \n\nIt's nice to meet you.  \n\nI'm here to help with any questions you have. What can I do for you today? 😄 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.000660939, 'queue_time': 0.020903039, 'total_time': 0.077024575}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1b4cb09f-4f8d-4c38-bb1a-9adcbb53c757-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Abhi")],
    config=config
)

response

AIMessage(content="Hi Abhi, it's nice to meet you! \n\nI'm happy to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.00031123, 'queue_time': 0.020549758, 'total_time': 0.08031123}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-008650ab-d22f-46b5-858a-98396905ea92-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Abhi!  I remember from our introduction.  How can I help you further? 😊  \n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhi")],"language":"Marathi"})
response.content

'नमस्कार अभि! मी तुमच्या मदतीसाठी येथे आहे. तुमच्या कोणत्याही प्रश्नासाठी विचार करा. 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am abhi")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते अभि! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप क्या जानना चाहते हैं? 😊\n'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [25]:
response.content

'आपका नाम अभि है, जैसा आपने खुद कहा। \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [34]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [29]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor! \n\nWhat's your favorite ice cream? 😊🍦  \n"

In [30]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"  😊 \n\n\n\nLet me know if you\'d like to try another one!\n'

In [31]:

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [32]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"